In [1]:
from __future__ import print_function, division

import numpy as np
import pandas as pd
import json, os, csv, shutil
from glob import glob
import matplotlib.pyplot as plt

In [2]:
import keras
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Lambda, Activation, Flatten, Input
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, RMSprop, SGD
from keras.utils import np_utils

import h5py
import cv2

/Users/swapnilpote/.virtualenvs/py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/swapnilpote/.virtualenvs/py3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
path = 'data/'

In [4]:
batch_size = 32

# Data preprocessing

In [5]:
dir_names = os.listdir(os.path.join(path, 'train/'))
dir_names

['NoF', 'OTHER', 'ALB', 'DOL', 'BET', 'LAG', 'YFT', 'SHARK']

In [6]:
dir_dict = {}

for i in range(len(dir_names)):
    dir_dict[dir_names[i]] = i

In [7]:
dir_dict

{'ALB': 2,
 'BET': 4,
 'DOL': 3,
 'LAG': 5,
 'NoF': 0,
 'OTHER': 1,
 'SHARK': 7,
 'YFT': 6}

In [8]:
X_train = []
y_train = []

In [9]:
for folder in os.listdir(path + 'train/'):
    for file in os.listdir(path + 'train/' + folder + '/'):
        if os.path.isfile(path + 'train/' + folder + '/' + file):
            
            input_img = cv2.imread(path + 'train/' + folder + '/' + file)
            input_img = cv2.resize(input_img, (224, 224))
            X_train.append(input_img)
            
            y_cat = np_utils.to_categorical(dir_dict[folder], 8) # 8 represent number of class
            y_train.append(y_cat)

In [10]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

In [11]:
print(X_train.shape)
print(y_train.shape)

(3025, 224, 224, 3)
(3025, 8)


In [12]:
y_train[466]

array([0., 1., 0., 0., 0., 0., 0., 0.])

In [13]:
X_valid = []
y_valid = []

In [14]:
for folder in os.listdir(path + 'valid/'):
    for file in os.listdir(path + 'valid/' + folder + '/'):
        if os.path.isfile(path + 'valid/' + folder + '/' + file):
            
            input_img = cv2.imread(path + 'valid/' + folder + '/' + file)
            input_img = cv2.resize(input_img, (224, 224))
            X_valid.append(input_img)
            
            y_cat = np_utils.to_categorical(dir_dict[folder], 8) # 8 represent number of class
            y_valid.append(y_cat)

In [15]:
X_valid = np.asarray(X_valid)
y_valid = np.asarray(y_valid)

In [16]:
print(X_valid.shape)
print(y_valid.shape)

(752, 224, 224, 3)
(752, 8)


In [17]:
y_valid[0]

array([1., 0., 0., 0., 0., 0., 0., 0.])

# BB

In [18]:
import ujson as json

In [19]:
anno_classes = ['alb', 'bet', 'dol', 'lag', 'other', 'shark', 'yft']

In [20]:
bb_json = {}

for c in anno_classes:
    j = json.load(open('{}annos/{}_labels.json'.format(path, c), 'r'))
    
    for l in j:
        if 'annotations' in l.keys() and len(l['annotations']) > 0:
            bb_json[l['filename']] = l['annotations'][-1]

In [21]:
bb_json['img_07763.jpg']

{'class': 'rect',
 'height': 127.00000000000045,
 'width': 121.00000000000045,
 'x': 636.0000000000023,
 'y': 353.00000000000125}

# NN

In [22]:
input_img = Input(shape=(224, 224, 3))

x = Conv2D(32, (3, 3), padding = 'same', activation = 'relu')(input_img)
# x = Conv2D(32, (3, 3), padding = 'same', activation = 'relu')(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(64, (3, 3), padding = 'same', activation = 'relu')(x)
# x = Conv2D(64, (3, 3), padding = 'same', activation = 'relu')(x)
x = MaxPooling2D((2, 2))(x)

x = Flatten()(x)
x = Dense(512, activation = 'relu')(x)
# x = Dense(512, activation = 'relu')(x)

x_bb = Dense(4, name='bb')(x)
x_class = Dense(8, activation='softmax', name='class')(x)

model = Model([input_img], [x_bb, x_class])
model.compile(Adam(lr=0.001), loss=['mse', 'categorical_crossentropy'], metrics=['accuracy'],
             loss_weights=[.001, 1.])

In [23]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 224, 224, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 112, 112, 32) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 112, 112, 64) 18496       max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
max_poolin

In [ ]:
model.fit(conv_feat, [trn_bbox, trn_labels], batch_size=batch_size, nb_epoch=3, 
             validation_data=(conv_val_feat, [val_bbox, val_labels]))